In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint
import os
RANDOM_STATE = 777

In [2]:
train_data = pd.read_csv(r"D:\PROJECTS\ML\5.2\ds\train.csv")
train_data.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Исключите признаки, которые на ваш взгляд, могут привести к переобучению (например, id). 

Разделите выборку на train и test.

In [4]:
train=train_data.drop([ 'PassengerId','Name','Ticket','Cabin','Embarked'], axis=1)# удаляю столбцы с именем, номером билета и каютой, портом посадки ( в столбце каюты - 204 значения всего)


Заполняем пропуски в столбце Age c помощью среднего значения возраста из группировки возрастов по классам обслуживания.

In [5]:
train['Age'] = train.groupby('Pclass')['Age'].transform(lambda x : x.fillna(x.mean()))

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 48.9+ KB


In [7]:
train.nunique()

Survived      2
Pclass        3
Sex           2
Age          91
SibSp         7
Parch         7
Fare        248
dtype: int64

Выделяем таргет и фичи:

In [8]:
X = train[['Pclass','Sex','Age', 'SibSp', 'Parch','Fare']]
y= train [['Survived']]


Делим на train и test c 30% данных в test.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = RANDOM_STATE)

Преобразуйте категориальные признаки с помощью sklearn.preprocessing.OneHotEncoder, а некатегориальные с помощью sklearn.preprocessing.StandardScaler

In [15]:
#вместо чудесного OHE есть встроенный в pandas метод getDummies. Делает то же самое, но гораздо проще

X_train=pd.get_dummies(X_train,drop_first='True')
X_test=pd.get_dummies(X_test,drop_first='True')

In [16]:
X_test

,Pclass,Age,SibSp,Parch,Fare,Sex_male
119,3,2.00000,4,2,31.2750,False
429,3,32.00000,0,0,8.0500,True
598,3,25.14062,0,0,7.2250,True
275,1,63.00000,1,0,77.9583,False
120,2,21.00000,2,0,73.5000,True
...,...,...,...,...,...,...
469,3,0.75000,2,1,19.2583,False
697,3,25.14062,0,0,7.7333,False
534,3,30.00000,0,0,8.6625,False
752,3,33.00000,0,0,9.5000,True


Теперь применяем StandartScaler

In [17]:
mms = StandardScaler()
X_train_std = mms.fit_transform(X_train)
X_train = pd.DataFrame(X_train_std, columns=X_train.columns)
X_test_std = mms.fit_transform(X_test)
X_test = pd.DataFrame(X_test_std, columns=X_test.columns)

In [19]:
X_test


,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,0.764654,-1.951121,3.255478,2.214140,-0.031873,-1.265678
1,0.764654,0.247003,-0.509217,-0.507830,-0.427796,0.790090
2,0.764654,-0.255589,-0.509217,-0.507830,-0.441860,0.790090
3,-1.734459,2.518399,0.431957,-0.507830,0.763952,-1.265678
4,-0.484903,-0.558976,1.373131,-0.507830,0.687950,0.790090
...,...,...,...,...,...,...
263,0.764654,-2.042710,1.373131,0.853155,-0.236725,-1.265678
264,0.764654,-0.255589,-0.509217,-0.507830,-0.433195,-1.265678
265,0.764654,0.100462,-0.509217,-0.507830,-0.417355,-1.265678
266,0.764654,0.320274,-0.509217,-0.507830,-0.403078,0.790090


## Логистическая регрессия

In [49]:
model = LogisticRegression()
model.fit(X_train,y_train.values.ravel())
y_pred = model.predict(X_test).reshape(-1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')


Accuracy: 0.7985074626865671
Precision: 0.7596153846153846
Recall: 0.7314814814814815
F1-score: 0.7452830188679245


## KNN

In [71]:
model = KNeighborsClassifier()
model.fit(X_train, y_train.values.ravel())
y_pred = model.predict(X_test).reshape(-1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')


Accuracy: 0.746268656716418
Precision: 0.717391304347826
Recall: 0.6111111111111112
F1-score: 0.6599999999999999


Точности моделей по долям правильных предсказаний (accuracy) схожи. 
По долям истинно положительных предсказаний среди всех положительных предсказаний (precision) лучше выглядит логистическая регрессия.
В это же время доля истинно положительных предсказаний среди всех истинных положительных случаев (recall) выше у логистической регрессии, что означает что в ответах регрессии точнее определена категория выжившего, т.е. какую долю истинно выживших точно предсказывает модель. У метода KNN этот параметр заметно хуже.
F-1 - гармоническая средняя между precision и recall. примерно одинаковая ввиду приблизительной одинаковости показателей моделей (recall у KNN заметно ниже при большом precision, у регрессии они римерно равны)

## RandomizedSearchCV для Логистической регрессии

In [66]:
model = LogisticRegression()

distributions = dict(
    penalty=['l2', 'l1', 'elasticnet', 'none'],
    C=uniform(loc=0, scale=4),
    fit_intercept=[True, False],
    solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    max_iter=[100, 200, 300, 400, 500]
)

clf = RandomizedSearchCV(model, distributions, n_iter=100, cv=5, verbose=0, n_jobs=-1)

search = clf.fit(X_train, y_train.values.ravel())

best_params = search.best_params_
search.best_params_

c:\Users\sever\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
205 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sever\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sever\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sever\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver =

{'C': 0.6486893427191278,
 'fit_intercept': True,
 'max_iter': 200,
 'penalty': 'l1',
 'solver': 'saga'}

In [67]:
model = LogisticRegression(solver=best_params['solver'], C=best_params['C'], penalty=best_params['penalty'], max_iter=best_params['max_iter'],fit_intercept=best_params['fit_intercept'])
model.fit(X_train, y_train.values.ravel())


y_pred = model.predict(X_test).reshape(-1)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')


Accuracy: 0.8022388059701493
Precision: 0.7722772277227723
Recall: 0.7222222222222222
F1-score: 0.7464114832535885


## RandomizedSearchCV для KNN

In [69]:

model = KNeighborsClassifier()

distributions = dict(
    n_neighbors=randint(1, 50),  # количество соседей
    weights=['uniform', 'distance'],  # функция весов
    algorithm=['auto', 'ball_tree', 'kd_tree', 'brute'],  # алгоритм, используемый для вычисления ближайших соседей
    leaf_size=randint(1, 50),
    p=[1, 2]  # параметр мощности для метрики Минковского
)

clf = RandomizedSearchCV(model, distributions, n_iter=100, cv=5, verbose=0, n_jobs=-1)


search = clf.fit(X_train, y_train.values.ravel())

best_params = search.best_params_
search.best_params_

c:\Users\sever\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [0.80585806 0.82672258 0.79784516 0.81553548 0.82030968 0.80103226
 0.80261935 0.80267097 0.80743226 0.80740645 0.79464516 0.82667097
 0.79624516 0.80587097 0.79781935 0.81710968 0.82348387        nan
 0.78504516 0.79781935        nan        nan 0.79785806 0.81227097
 0.79944516 0.82196129 0.79619355        nan 0.80104516 0.79624516
 0.80905806 0.80264516        nan 0.80261935 0.78823226        nan
 0.80419355 0.81545806 0.80744516        nan        nan 0.80584516
 0.80263226 0.81069677 0.80101935 0.82029677        nan 0.80263226
 0.80419355        nan 0.80588387 0.82188387 0.79145806        nan
 0.79787097 0.79945806        nan 0.82185806 0.79941935 0.80263226
 0.81545806 0.80107097 0.79464516 0.79784516 0.81384516 0.79619355
 0.82356129 0.79784516 0.80263226 0.78983226 0.80588387 0.80104516
 0.80265806        nan 0.80421935        nan 0.807458

{'algorithm': 'auto',
 'leaf_size': 44,
 'n_neighbors': 21,
 'p': 1,
 'weights': 'uniform'}

In [70]:
model = KNeighborsClassifier(algorithm=best_params['algorithm'], leaf_size =best_params['leaf_size'],n_neighbors=best_params['n_neighbors'],p=best_params['p'],weights=best_params['weights'])
model.fit(X_train, y_train.values.ravel())
y_pred = model.predict(X_test).reshape(-1)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')


Accuracy: 0.8059701492537313
Precision: 0.8255813953488372
Recall: 0.6574074074074074
F1-score: 0.7319587628865979


## ИТОГИ
Использование случайного поиска гиперпараметров улучшило модель KNN что говорит о важности используемых параметров и их настройке при импользовании данного метода. Для линейной регрессии использование RandomizedGridSearchCV не дало существенного изменения метрик точности. В целом результы хоть и неменого улучшились, но по метрике recall KNN все равно остается слабее логистической регрессии. Так же важно заметить якобы высокий показатель accuracy и одновременно низкий покзатель recall - что говорит о недостаточности лишь одного значения accuracy для оценки и интерпретации работы модели в целом (в тесте был вопрос об этом но я ответил неверно - в чем каюсь, да и про recall в тесте  я тоже мимо ответил - теперь вспомнил о чем этот показатель и стыдно )